In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.display import HTML
with open("rise.css", "r") as f:
    s = f"<style>{f.read()}</style>"
HTML(s)

In [ ]:
import plotly.graph_objects as go
from copy import deepcopy
import plotly.io as pio

my_template = deepcopy(pio.templates["plotly_dark"])
my_template["layout"]["colorway"] = ["#D01D18", "#0091CF", "#00CF9A"]
my_template["layout"]["paper_bgcolor"] = "#131424"
my_template["layout"]["plot_bgcolor"] = "#131424"
my_template["layout"]["font"]["color"] = "#fffac9"
pio.templates["notrocket"] = my_template
pio.templates.default = "notrocket"


<div class="title">

<h1 class="title">Not Rocketscience</h1>

## A spaceship-flying game built with pygame

### Andreas Roth - 13. June 2024

</div>


## Gravity
    
not newton.
    


In [ ]:
from not_rocketscience.math import weird_gravity_force as deriv_combination, weird_gravity_potential as combination
import numpy as np

x = np.linspace(0, 2, 100)
y1 = -combination(4, 6, 0.2, 4, 1.2)(x)
y2 = -deriv_combination(4, 6, 0.2, 4, 1.2)(x)
go.Figure(
    data=[
        go.Scatter(name="Gravity potential", x=x, y=y1), go.Scatter(name="Gravity force", x=x, y=y2)
    ],
    layout=dict(height=300, title="Gravity potential and force over distance", xaxis=dict(title="distance"))
)


In [ ]:
from not_rocketscience.math import newton_gravity

go.Figure(data=[go.Scatter(name="Newton", x=x, y=newton_gravity(x))])